In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests 
import random
import json
!pip install yfinance
import yfinance as yf
import numpy as np 

  Using cached yfinance-0.2.41-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached frozendict-2.4.4-py311-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.6-cp311-cp311-linux_x86_64.whl
Using cached yfinance-0.2.41-py2.py3-none-any.whl (73 kB)
Using cached frozendict-2.4.4-py311-none-any.whl (16 kB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)


In [3]:
def get_bs_from_ticker(ticker: str, table: bool=True, plot: bool=True):
    headers = {'User-Agent': "arnavdey02@gmail.com"}

    all_tickers = requests.get(
                                "https://www.sec.gov/files/company_tickers.json",
                                headers=headers
                                )
    company_data = pd.DataFrame.from_dict(all_tickers.json(),
                                     orient='index')
    company_data['cik_str'] = company_data['cik_str'].astype(str).str.zfill(10)
    
    try: 
        my_cik = company_data[company_data['ticker'] == ticker.upper()]['cik_str'].iloc[0]
    except:
        print(f"Ticker {ticker} not found in database.")
        return 
    
    root = f'https://data.sec.gov/api/xbrl/companyconcept/CIK{my_cik}/us-gaap'
    urls = {
        "total_assets": f'{root}/Assets.json',
        "total_liabilities": f'{root}/Liabilities.json',
        "stockholders_equity": f'{root}/StockholdersEquity.json',
        "current_assets": f'{root}/AssetsCurrent.json',
        "current_liabilities": f'{root}/LiabilitiesCurrent.json',
        # "revenues": f'{root}/Revenues.json',
        "net_income_loss": f'{root}/NetIncomeLoss.json'
    }
    
    data = {}
    for metric, url in urls.items():
        response = requests.get(url, headers=headers)
        data[metric] = pd.DataFrame.from_dict(response.json()['units']['USD'])
        
    def process_metric(metric_data, key):
        metric_data = metric_data[metric_data.form == '10-Q']
        metric_data = metric_data.reset_index(drop=True)
        metric_data = metric_data[['end', 'val']]
        metric_data['val'] = metric_data['val'] / 1e0
        metric_data.columns = ["Date",f"{str(key)}"]
        # metric_data.set_index('Date', inplace=True)
        return metric_data
    lengths = []
    for key, value in data.items():
        data[key] = process_metric(value, key)
        lengths.append(data[key].shape[0])
        # data[key] = data[key].reset_index()

    first_key = list(data.keys())[0]
    merged_df = data[first_key].copy()
    # print(merged_df)
    for key in list(data.keys())[1:]:
        df = data[key]
        merged_df = pd.merge(merged_df, df, on='Date', how='outer', suffixes=('', f'_{key}'))
    merged_df['Date'] = pd.to_datetime(merged_df['Date'])
    merged_df['Year'] = merged_df['Date'].dt.year
    merged_df['Quarter'] = merged_df['Date'].dt.quarter
    merged_df = merged_df.set_index('Date')
    merged_df = merged_df.ffill().bfill()
    merged_df = merged_df[~merged_df.index.duplicated(keep='first')]
    
    
    pd.set_option('display.float_format', '{:.2e}'.format)
    print(f"Balance Sheet Metrics for ticker {ticker}")
    merged_df.sort_index(axis=1, ascending=True)


    return merged_df

In [4]:
df1 = get_bs_from_ticker('AAPL')
df1.head()

Balance Sheet Metrics for ticker AAPL


,total_assets,total_liabilities,stockholders_equity,current_assets,current_liabilities,net_income_loss,Year,Quarter
Date,,,,,,,,
2008-09-27,3.96e+10,1.85e+10,2.10e+10,3.23e+10,1.41e+10,4.04e+09,2008,3
2009-06-27,4.81e+10,2.23e+10,2.59e+10,3.52e+10,1.67e+10,4.04e+09,2009,2
2009-09-26,4.75e+10,1.59e+10,3.16e+10,3.16e+10,1.15e+10,1.83e+09,2009,3
2009-12-26,5.39e+10,1.82e+10,3.58e+10,3.33e+10,1.31e+10,3.38e+09,2009,4
2010-03-27,5.71e+10,1.77e+10,3.93e+10,3.23e+10,1.22e+10,6.45e+09,2010,1


In [5]:

def time_frame(data, year:int, quarter:int):
    df2 = data[data['Year']== year]
    df2 = df2[df2['Quarter'] == quarter]
    return df2
    
    
time_frame(df1,2010,4)


,total_assets,total_liabilities,stockholders_equity,current_assets,current_liabilities,net_income_loss,Year,Quarter
Date,,,,,,,,
2010-12-25,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4


In [6]:
# creating a list of S&P500 tickers
tickers = pd.read_csv('stock_tickers.csv')
ticker_symbols = list(tickers['ticker'])


# Select n random tickers
#This is sampling without replacement which is what we want
#random_tickers = random.sample(ticker_symbols, 100)
random_tickers = [
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corp.
    'AMZN',  # Amazon.com Inc.
    'GOOGL', # Alphabet Inc. (Class A)
    'GOOG',  # Alphabet Inc. (Class C)
    'BRK.B', # Berkshire Hathaway Inc. (Class B)
    'NVDA',  # NVIDIA Corp.
    'TSLA',  # Tesla Inc.
    'META',  # Meta Platforms Inc.
    'UNH',   # UnitedHealth Group Inc.
    'JNJ',   # Johnson & Johnson
    'V',     # Visa Inc.
    'XOM',   # Exxon Mobil Corp.
    'PG',    # Procter & Gamble Co.
    'JPM',   # JPMorgan Chase & Co.
    'MA',    # Mastercard Inc.
    'HD',    # Home Depot Inc.
    'MRK',   # Merck & Co. Inc.
    'ABBV',  # AbbVie Inc.
    'PFE',   # Pfizer Inc.
    'PEP',   # PepsiCo Inc.
    'KO',    # Coca-Cola Co.
    'LLY',   # Eli Lilly and Co.
    'AVGO',  # Broadcom Inc.
    'TMO',   # Thermo Fisher Scientific Inc.
    'COST',  # Costco Wholesale Corp.
    'CSCO',  # Cisco Systems Inc.
    'MCD',   # McDonald's Corp.
    'ABT',   # Abbott Laboratories
    'ACN',   # Accenture plc
    'DHR',   # Danaher Corp.
    'TXN',   # Texas Instruments Inc.
    'NEE',   # NextEra Energy Inc.
    'VZ',    # Verizon Communications Inc.
    'DIS',   # Walt Disney Co.
    'CRM',   # Salesforce Inc.
    'WMT',   # Walmart Inc.
    'PM',    # Philip Morris International Inc.
    'ADBE',  # Adobe Inc.
    'NFLX',  # Netflix Inc.
    'NKE',   # Nike Inc.
    'LIN',   # Linde plc
    'AMD',   # Advanced Micro Devices Inc.
    'MDT',   # Medtronic plc
    'LOW',   # Lowe's Companies Inc.
    'ORCL',  # Oracle Corp.
    'INTC',  # Intel Corp.
    'HON',   # Honeywell International Inc.
    'UNP',   # Union Pacific Corp.
    'UPS'    # United Parcel Service Inc.
]

print(random_tickers)

['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'BRK.B', 'NVDA', 'TSLA', 'META', 'UNH', 'JNJ', 'V', 'XOM', 'PG', 'JPM', 'MA', 'HD', 'MRK', 'ABBV', 'PFE', 'PEP', 'KO', 'LLY', 'AVGO', 'TMO', 'COST', 'CSCO', 'MCD', 'ABT', 'ACN', 'DHR', 'TXN', 'NEE', 'VZ', 'DIS', 'CRM', 'WMT', 'PM', 'ADBE', 'NFLX', 'NKE', 'LIN', 'AMD', 'MDT', 'LOW', 'ORCL', 'INTC', 'HON', 'UNP', 'UPS']


In [7]:


df2 = time_frame(df1,2010,4)
df2['ticker'] = 'AAPL'

def create_dataframe(data, tickers:list,year:int,quarter:int):
    for ticker in tickers:
        try:
            # Assuming get_bs_from_ticker can raise JSONDecodeError
            dfs = get_bs_from_ticker(ticker)
            # Assuming time_frame can also raise JSONDecodeError or other errors
            df_to_add = time_frame(dfs, year, quarter)
            df_to_add['ticker'] = ticker
            data = pd.concat([data, df_to_add], ignore_index=True)
        except json.JSONDecodeError as e:
            print(f"JSON decode error for ticker {ticker}: {e}")
        except Exception as e:
            print(f"An error occurred for ticker {ticker}: {e}")
    return data

balance_sheet_df = create_dataframe(df2,random_tickers,2010,4)
    
    

Balance Sheet Metrics for ticker AAPL
Balance Sheet Metrics for ticker MSFT
JSON decode error for ticker AMZN: Expecting value: line 1 column 1 (char 0)
Balance Sheet Metrics for ticker GOOGL
Balance Sheet Metrics for ticker GOOG
Ticker BRK.B not found in database.
An error occurred for ticker BRK.B: 'NoneType' object is not subscriptable
Balance Sheet Metrics for ticker NVDA
Balance Sheet Metrics for ticker TSLA
Balance Sheet Metrics for ticker META
Balance Sheet Metrics for ticker UNH
Balance Sheet Metrics for ticker JNJ
Balance Sheet Metrics for ticker V
Balance Sheet Metrics for ticker XOM
JSON decode error for ticker PG: Expecting value: line 1 column 1 (char 0)
JSON decode error for ticker JPM: Expecting value: line 1 column 1 (char 0)
Balance Sheet Metrics for ticker MA
Balance Sheet Metrics for ticker HD
JSON decode error for ticker MRK: Expecting value: line 1 column 1 (char 0)
JSON decode error for ticker ABBV: Expecting value: line 1 column 1 (char 0)
Balance Sheet Metrics f

In [8]:
balance_sheet_df

,total_assets,total_liabilities,stockholders_equity,current_assets,current_liabilities,net_income_loss,Year,Quarter,ticker
0,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL
1,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL
2,9.23e+10,4.38e+10,4.85e+10,5.97e+10,2.43e+10,1.20e+10,2010,4,MSFT
3,3.92e+09,2.81e+09,2.90e+09,2.81e+09,8.16e+08,8.15e+07,2010,4,NVDA
4,3.86e+08,1.79e+08,2.07e+08,2.36e+08,8.56e+07,-1.08e+08,2010,4,TSLA
5,6.31e+10,3.72e+10,2.58e+10,1.84e+10,2.37e+10,1.28e+09,2010,4,UNH
6,9.82e+10,4.10e+10,7.68e+10,4.27e+10,1.74e+10,9.45e+09,2010,4,JNJ
7,3.40e+10,8.44e+09,2.55e+10,9.36e+09,3.68e+09,8.84e+08,2010,4,V
8,3.03e+11,1.50e+11,1.47e+11,5.90e+10,6.26e+10,7.35e+09,2010,4,XOM
9,8.84e+09,3.62e+09,5.20e+09,6.45e+09,3.14e+09,5.18e+08,2010,4,MA


- So the function ignores the JSON decode error and continues the program (error occurs due to mismatch between tickers in JSON and table which I don't know how)
- It is still missing the last three stocks likely because all of them are missing one or more of the balance sheet metrics we want and so they are not included

In [9]:
#Now to get quarterly prices data for the stocks and calculating quarterly returns
stock_price_data = []

for ticker in random_tickers:
    data = yf.download(ticker, start='2010-01-01', end='2020-01-01',interval='1mo')['Adj Close']
    stock_price_data.append(data)

    


# Combine individual series into a DataFrame
stock_price_data_full = pd.concat(stock_price_data, axis=1)
stock_price_data_full.columns = [ticker for ticker in random_tickers]
stock_price_data_full = stock_price_data_full.dropna(axis=1, how='any')


stock_price_data_full = stock_price_data_full.reset_index()
stock_price_data_full['Date'] = pd.to_datetime(stock_price_data_full['Date'])
stock_price_data_full.set_index('Date', inplace=True)

stock_price_quarterly = stock_price_data_full.resample('Q').last()
    
stock_price_quarterly.head()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['GOOG']: OperationalError('disk I/O error')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['META']: OperationalError('disk I/O error')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed do

,AAPL,MSFT,AMZN,GOOGL,NVDA,UNH,V,PG,MA,HD,...,NFLX,NKE,LIN,AMD,MDT,LOW,INTC,HON,UNP,UPS
Date,,,,,,,,,,,,,,,,,,,,,
2010-03-31,7.10e+00,2.22e+01,6.79e+00,1.42e+01,3.99e-01,2.63e+01,2.05e+01,4.14e+01,2.35e+01,2.29e+01,...,1.05e+01,1.54e+01,6.25e+01,9.27e+00,3.21e+01,1.86e+01,1.46e+01,3.17e+01,2.70e+01,4.16e+01
2010-06-30,7.59e+00,1.75e+01,5.46e+00,1.11e+01,2.34e-01,2.29e+01,1.60e+01,3.95e+01,1.85e+01,2.00e+01,...,1.55e+01,1.42e+01,5.75e+01,7.32e+00,2.59e+01,1.57e+01,1.29e+01,2.75e+01,2.57e+01,3.70e+01
2010-09-30,8.57e+00,1.88e+01,7.85e+00,1.31e+01,2.68e-01,2.84e+01,1.68e+01,3.98e+01,2.08e+01,2.29e+01,...,2.32e+01,1.69e+01,6.87e+01,7.11e+00,2.42e+01,1.73e+01,1.28e+01,3.12e+01,3.04e+01,4.36e+01
2010-12-31,9.74e+00,2.15e+01,9.00e+00,1.48e+01,3.53e-01,2.93e+01,1.60e+01,4.30e+01,2.08e+01,2.56e+01,...,2.51e+01,1.81e+01,7.31e+01,8.18e+00,2.69e+01,1.95e+01,1.41e+01,3.80e+01,3.46e+01,4.78e+01
2011-03-31,1.05e+01,1.97e+01,9.01e+00,1.47e+01,4.23e-01,3.68e+01,1.67e+01,4.15e+01,2.34e+01,2.70e+01,...,3.40e+01,1.61e+01,7.81e+01,8.60e+00,2.87e+01,2.07e+01,1.37e+01,4.29e+01,3.69e+01,4.93e+01


In [10]:
returns = stock_price_quarterly.pct_change(fill_method=None).dropna()
returns = returns.reset_index()
returns['Date'] = pd.to_datetime(returns['Date'])
returns['Year'] = returns['Date'].dt.year
returns['Quarter'] = returns['Date'].dt.quarter
returns.head()

,Date,AAPL,MSFT,AMZN,GOOGL,NVDA,UNH,V,PG,MA,...,LIN,AMD,MDT,LOW,INTC,HON,UNP,UPS,Year,Quarter
0,2010-06-30,7.03e-02,-2.11e-01,-1.95e-01,-2.15e-01,-4.13e-01,-1.30e-01,-2.22e-01,-4.47e-02,-2.14e-01,...,-7.91e-02,-2.10e-01,-1.91e-01,-1.55e-01,-1.21e-01,-1.32e-01,-4.71e-02,-1.11e-01,2010,2
1,2010-09-30,1.28e-01,7.00e-02,4.37e-01,1.82e-01,1.44e-01,2.41e-01,5.14e-02,7.66e-03,1.23e-01,...,1.95e-01,-2.87e-02,-6.84e-02,9.76e-02,-5.32e-03,1.34e-01,1.82e-01,1.81e-01,2010,3
2,2010-12-31,1.37e-01,1.47e-01,1.46e-01,1.30e-01,3.18e-01,3.23e-02,-5.03e-02,8.09e-02,1.16e-03,...,6.32e-02,1.50e-01,1.12e-01,1.31e-01,1.04e-01,2.17e-01,1.38e-01,9.58e-02,2010,4
3,2011-03-31,8.04e-02,-8.49e-02,7.22e-04,-1.21e-02,1.99e-01,2.56e-01,4.81e-02,-3.53e-02,1.24e-01,...,6.93e-02,5.13e-02,6.74e-02,5.86e-02,-3.23e-02,1.30e-01,6.56e-02,3.12e-02,2011,1
4,2011-06-30,-3.68e-02,3.07e-02,1.35e-01,-1.37e-01,-1.37e-01,1.44e-01,1.47e-01,4.05e-02,1.98e-01,...,7.23e-02,-1.87e-01,-1.52e-02,-1.14e-01,1.07e-01,3.66e-03,6.67e-02,-1.18e-02,2011,2


In [11]:
desired_return = pd.DataFrame(returns[(returns['Quarter'] == 4) & (returns['Year']== 2010)].iloc[-1])
desired_return = desired_return.reset_index()
desired_return = desired_return.iloc[1:32]
desired_return.columns = ['stock','% return from quarter']
desired_return

,stock,% return from quarter
1,AAPL,1.37e-01
2,MSFT,1.47e-01
3,AMZN,1.46e-01
4,GOOGL,1.30e-01
5,NVDA,3.18e-01
6,UNH,3.23e-02
7,V,-5.03e-02
8,PG,8.09e-02
9,MA,1.16e-03
10,HD,1.15e-01


In [12]:
merged_df = pd.merge(balance_sheet_df, desired_return, left_on = 'ticker', right_on = 'stock')

In [13]:
merged_df

,total_assets,total_liabilities,stockholders_equity,current_assets,current_liabilities,net_income_loss,Year,Quarter,ticker,stock,% return from quarter
0,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL,AAPL,1.37e-01
1,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL,AAPL,1.37e-01
2,9.23e+10,4.38e+10,4.85e+10,5.97e+10,2.43e+10,1.20e+10,2010,4,MSFT,MSFT,1.47e-01
3,3.92e+09,2.81e+09,2.90e+09,2.81e+09,8.16e+08,8.15e+07,2010,4,NVDA,NVDA,3.18e-01
4,6.31e+10,3.72e+10,2.58e+10,1.84e+10,2.37e+10,1.28e+09,2010,4,UNH,UNH,3.23e-02
5,3.40e+10,8.44e+09,2.55e+10,9.36e+09,3.68e+09,8.84e+08,2010,4,V,V,-5.03e-02
6,8.84e+09,3.62e+09,5.20e+09,6.45e+09,3.14e+09,5.18e+08,2010,4,MA,MA,1.16e-03
7,4.17e+10,2.26e+10,1.91e+10,1.51e+10,1.14e+10,2.75e+09,2010,4,HD,HD,1.15e-01
8,1.91e+11,1.03e+11,8.76e+10,5.41e+10,2.42e+10,5.37e+09,2010,4,PFE,PFE,3.03e-02
9,1.95e+11,1.07e+11,8.78e+10,6.10e+10,2.86e+10,8.66e+08,2010,4,PFE,PFE,3.03e-02


In [14]:
#getting market cap and producing regression weights
market_caps = {}
for ticker in random_tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    if 'marketCap' in info:
        market_caps[ticker] = info['marketCap']
    else:
        market_caps[ticker] = None
        print(f"Market cap data not available for {ticker}")
type(market_caps)


Market cap data not available for BRK.B


dict

In [37]:
market_caps_list = {key: [value] for key, value in market_caps.items()}
market_caps_df = pd.DataFrame(market_caps_list)
market_caps_df = market_caps_df.transpose()
market_caps_df['market cap'] = market_caps_df[0]
market_caps_df = market_caps_df.dropna()

market_cap_values = list(market_caps_df['market cap'])


def weights(values):
    summed = 0
    for i in values:
        summed = summed + i
    weight = []
    for j in values:
        weight.append((np.sqrt(j))/np.sqrt(summed))
    return weight

market_caps_df['weight'] = weights(market_cap_values)
market_caps_df = market_caps_df.drop(columns=0)
market_caps_df = market_caps_df.reset_index()
market_caps_df.columns = ['stocks','market cap','weight']
market_caps_df

    

,stocks,market cap,weight
0,AAPL,3181762052096,3.40e-01
1,MSFT,2937165709312,3.26e-01
2,AMZN,1690001473536,2.48e-01
3,GOOGL,1969651843072,2.67e-01
4,GOOG,1966313832448,2.67e-01
5,NVDA,2470899351552,2.99e-01
6,TSLA,635350024192,1.52e-01
7,META,1203511230464,2.09e-01
8,UNH,524582682624,1.38e-01
9,JNJ,388167434240,1.19e-01


In [39]:
merged_df_final = pd.merge(merged_df, market_caps_df, left_on = 'ticker', right_on = 'stocks')
merged_df_final.head()

,total_assets,total_liabilities,stockholders_equity,current_assets,current_liabilities,net_income_loss,Year,Quarter,ticker,stock,% return from quarter,stocks,market cap,weight
0,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL,AAPL,1.37e-01,AAPL,3181762052096,3.40e-01
1,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL,AAPL,1.37e-01,AAPL,3181762052096,3.40e-01
2,9.23e+10,4.38e+10,4.85e+10,5.97e+10,2.43e+10,1.20e+10,2010,4,MSFT,MSFT,1.47e-01,MSFT,2937165709312,3.26e-01
3,3.92e+09,2.81e+09,2.90e+09,2.81e+09,8.16e+08,8.15e+07,2010,4,NVDA,NVDA,3.18e-01,NVDA,2470899351552,2.99e-01
4,6.31e+10,3.72e+10,2.58e+10,1.84e+10,2.37e+10,1.28e+09,2010,4,UNH,UNH,3.23e-02,UNH,524582682624,1.38e-01


In [40]:
merged_df_final['intercept'] = 1
merged_df_final.head()

,total_assets,total_liabilities,stockholders_equity,current_assets,current_liabilities,net_income_loss,Year,Quarter,ticker,stock,% return from quarter,stocks,market cap,weight,intercept
0,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL,AAPL,1.37e-01,AAPL,3181762052096,3.40e-01,1
1,8.67e+10,3.21e+10,5.47e+10,4.39e+10,2.38e+10,6.00e+09,2010,4,AAPL,AAPL,1.37e-01,AAPL,3181762052096,3.40e-01,1
2,9.23e+10,4.38e+10,4.85e+10,5.97e+10,2.43e+10,1.20e+10,2010,4,MSFT,MSFT,1.47e-01,MSFT,2937165709312,3.26e-01,1
3,3.92e+09,2.81e+09,2.90e+09,2.81e+09,8.16e+08,8.15e+07,2010,4,NVDA,NVDA,3.18e-01,NVDA,2470899351552,2.99e-01,1
4,6.31e+10,3.72e+10,2.58e+10,1.84e+10,2.37e+10,1.28e+09,2010,4,UNH,UNH,3.23e-02,UNH,524582682624,1.38e-01,1


In [41]:
merged_df_final.to_csv('cross-section-data.csv',index=False)